In [62]:
import pandas as pd
import numpy as np
from modeling import Transfer_Model
import tensorflow as tf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


from tensorflow.keras.metrics import RootMeanSquaredError

# Variables from config file
import sys
sys.path.append('/home/yusukemh/github/yusukemh/StatisticalDownscaling/codes/')
from config import BASE_DIR, C_SINGLE, C_GRID, C_COMMON
# enable autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
def assign_fold(df, n_folds = 5):
    df_n_data = df_nonfilled.groupby('skn').size().reset_index().rename(columns={0: "n_data"})
    df_n_data = df_n_data[df_n_data['n_data'] > 750]
    df = df.merge(df_n_data, left_on='skn', right_on='skn', how='inner')
    
    df_len_by_month = pd.DataFrame(df.groupby(by=['year', 'month']).size()).reset_index().rename({0: "len"}, axis=1)
    df_len_by_month['cumsum'] = df_len_by_month['len'].cumsum()
    n_samples_total = df_len_by_month['cumsum'].iloc[-1]
    n_samples_per_fold = np.ceil(n_samples_total / n_folds)
    
    df_len_by_month['fold'] = df_len_by_month.apply(lambda row: int(row['cumsum'] / n_samples_per_fold), axis=1)
    
    df_w_fold = pd.merge(left=df, right=df_len_by_month, left_on=['year', 'month'], right_on=['year', 'month'])
    
    return df_w_fold

In [21]:
df_nonfilled = pd.read_csv(f"{BASE_DIR}/nonfilled_dataset.csv", usecols=C_SINGLE + C_COMMON)
df_filtered = assign_fold(df_nonfilled)

In [32]:
model = Model(inputs=C_SINGLE, outputs='data_in')
model.set_model(
    dict(
        input_dim=len(C_SINGLE),
        lr=0.0001,
        activation='elu',
        n_units=216,
        n_layers=0,
        dropout=0.5
    )
)
x_train, x_test, y_train, y_test = model.train_test_split(df_filtered, fold=1, fine_tune=False)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
history_pretraining = model.model.fit(
    x_train, y_train,
    epochs=2000,
    verbose=True,
    callbacks=[callback],
    validation_split=0.2
)

Epoch 1/1000
370/370 [==============================] - 1s 1ms/step - loss: 35.8100 - root_mean_squared_error: 5.9841 - val_loss: 33.7840 - val_root_mean_squared_error: 5.8124
Epoch 2/1000
370/370 [==============================] - 1s 1ms/step - loss: 30.5277 - root_mean_squared_error: 5.5252 - val_loss: 32.4226 - val_root_mean_squared_error: 5.6941
Epoch 3/1000
370/370 [==============================] - 0s 1ms/step - loss: 29.4879 - root_mean_squared_error: 5.4303 - val_loss: 31.2412 - val_root_mean_squared_error: 5.5894
Epoch 4/1000
370/370 [==============================] - 0s 1ms/step - loss: 28.5904 - root_mean_squared_error: 5.3470 - val_loss: 30.3206 - val_root_mean_squared_error: 5.5064
Epoch 5/1000
370/370 [==============================] - 0s 1ms/step - loss: 27.8894 - root_mean_squared_error: 5.2810 - val_loss: 29.5583 - val_root_mean_squared_error: 5.4368
Epoch 6/1000
370/370 [==============================] - 0s 1ms/step - loss: 27.2899 - root_mean_squared_error: 5.2240 - 

In [34]:
model.model.save('models/testing.hdf5')

In [36]:
skn = 79
trained_model = tf.keras.models.load_model('models/testing.hdf5')

In [46]:
transfer_model = Transfer_Model(inputs=C_SINGLE, outputs='data_in')
x_train, y_train, x_test, y_test, x_train_station, x_test_station, y_train_station, y_test_station = \
transfer_model.train_test_split(df_filtered, fold=1, fine_tune=True, skn=79)

In [56]:
trained_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.000005),
    loss='mse',
    metrics=[RootMeanSquaredError()]
)
callback_finetune = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)


In [57]:
history_finetune = trained_model.fit(x_train_station, y_train_station, epochs=500, verbose=2, callbacks=[callback_finetune], validation_split=0.2)

Epoch 1/500
16/16 - 0s - loss: 33.6996 - root_mean_squared_error: 5.8051 - val_loss: 31.3638 - val_root_mean_squared_error: 5.6003 - 387ms/epoch - 24ms/step
Epoch 2/500
16/16 - 0s - loss: 33.6366 - root_mean_squared_error: 5.7997 - val_loss: 31.2871 - val_root_mean_squared_error: 5.5935 - 40ms/epoch - 2ms/step
Epoch 3/500
16/16 - 0s - loss: 33.5758 - root_mean_squared_error: 5.7945 - val_loss: 31.2256 - val_root_mean_squared_error: 5.5880 - 38ms/epoch - 2ms/step
Epoch 4/500
16/16 - 0s - loss: 33.5355 - root_mean_squared_error: 5.7910 - val_loss: 31.1896 - val_root_mean_squared_error: 5.5848 - 35ms/epoch - 2ms/step
Epoch 5/500
16/16 - 0s - loss: 33.4857 - root_mean_squared_error: 5.7867 - val_loss: 31.1140 - val_root_mean_squared_error: 5.5780 - 54ms/epoch - 3ms/step
Epoch 6/500
16/16 - 0s - loss: 33.4264 - root_mean_squared_error: 5.7816 - val_loss: 31.0186 - val_root_mean_squared_error: 5.5694 - 36ms/epoch - 2ms/step
Epoch 7/500
16/16 - 0s - loss: 33.3623 - root_mean_squared_error: 5.

In [64]:


yhat_station = trained_model.predict(x_test_station)
rmse_nn = mean_squared_error(y_test_station, yhat_station, squared=False)
# rmse_nn = mean_squared_error(np.power(np.e, Ytest_station), np.power(np.e, yhat_station), squared=False)

# Linear Regression
linear_regression = LinearRegression()
linear_regression.fit(x_train_station, y_train_station)
yhat_linear_regression = linear_regression.predict(x_test_station)
rmse_lr = mean_squared_error(y_test_station, yhat_linear_regression, squared=False)
# rmse_lr = mean_squared_error(np.power(np.e, Ytest_station), np.power(np.e, yhat_linear_regression), squared=False)

print('Fold {}\n LR: {:.10f}, NN: {:.10f}'.format(1, rmse_lr, rmse_nn))

Fold 1
 LR: 5.0894674478, NN: 5.1425869715
